In [0]:
from pyspark.sql import functions as F
from delta.tables import *

TABELA_ORIGEM = "v_credit.silver.tb_motivo"
TABELA_DESTINO = "v_credit.gold.dm_motivo"

In [0]:
df_silver = spark.read.table(TABELA_ORIGEM)

df_gold = df_silver.select(
    F.col("cd_motivo"),
    F.col("ds_motivo"),
    F.col("ds_categoria"),
    F.col("ds_criticidade"),
    
    F.when(
        F.upper(F.col("ds_criticidade")).like("%BAIXA%"), "Alto Potencial Automação"
    ).when(
        (F.upper(F.col("ds_criticidade")).like("%MEDIA%")) | (F.upper(F.col("ds_criticidade")).like("%MÉDIA%")), "Médio Potencial"
    ).otherwise("Baixo (Requer Humano)").alias("ds_potencial_automacao"),
    
    F.current_timestamp().alias("dt_ingestion"),
    F.lit("ETL_Databricks_Merge").alias("dc_origem")
)

deltaTable = DeltaTable.forName(spark, TABELA_DESTINO)

deltaTable.alias("target").merge(
    df_gold.alias("source"),
    "target.cd_motivo = source.cd_motivo"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Merge realizado com sucesso em {TABELA_DESTINO}")